### Required Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Positional Encoding

In [2]:
"""
Because the network eliminates convolutional layers, and recurrent layers it must have a way to understand the position
of words in respect to one another. Saying 'I saw a movie.' is not the same as 'Movie a I saw.' In order to do this
sine and cosine is used to correspond each dimension of the positional encoding to a sinusoid wave. Note dimensions of
encodings share same dimensions as d model so they can be summed together. (i.e. d model = embedding dimensions). Also,
some inputs are dropped to reduce overfitting when passing inputs from the multi-head attention sub-layer.
"""

class PositionalEncoder(nn.Module):
    def __init__(self, d_model, dropout, maxlen):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        
        # Positional Encoding Sinusoid Formula
        pos_encoding = torch.zeros(maxlen, d_model)
        pos = torch.arange(0, maxlen, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5
        denom = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0)) / d_model) # 1000^(2i/d_model)        
        pos_encoding[:, 0::2] = torch.sin(pos * denom)
        pos_encoding[:, 1::2] = torch.cos(pos * denom)
        
        # saving encoingss without gradients
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pos_encoding', pos_encoding)
        
    def forward(self, token_embedding):
        # Residual connection + positional encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

### Transformer Network

In [3]:
"""
Transformer network architecture declaration. The basic architecture requires the number of tokens (i.e. total amount of unique words),
d model described as the dimensions the encoder/decoder expects as inputs (embedding projections), number of heads which identifies 
the number of heads used for the multi-head attention model (i.e. number of linear projections for Q, K, & V), number of encoder layers
which decides the amount of multi-head attention (normed) & feed forward (normed) sequences, number of decoder layers which entails the 
total amount of masked multi-head attention (normed), multi-head attention (normed), & feed forward (norm) sequences, feed forward which describes
the amount of hidden dimensions to in the feed forward network, and lastly dropout which indicates the percentage of inputs to be dropped
when passing to another sub-layer (i.e. percent of inputs to be dropped in masked mult-head attention, multi-head attention, & feed forward
netowrk).


Encoder
Inputs are fed into the embedding layer of the encoder where the tokenized inputs are transformed into vectors of values with the dimensions
equivalent to d model. Next the embeddings are positionaly encoded as described in the PositionalEncoding class above. Results from this
layer are dropped (based on dropout percentage) and passed to the 1st & 2nd sub-layer of the encoder through residual connections.

The multi-head attention layer (1st sub-layer) recieves the positional encodings and computes the weighted similarity of values based on
the compatability function between the queries & keys for each value (i.e. weighted a word based on similarities with other words). These
are concatenated into h (h being the number of heads) learnable linear projections which are then normalized.

The feed forward network (2nd sub-layer) recieves these normalized projections and processes it such that it fits better
to the next multi-head attention layer. This result is normalized and passed to the unmasked multi-head attention layer in
the decoder.

Decoder
The decoder takes the output (input from encoder shifted right) embeds it, then positionaly encodes it (described above). 
As mentioned the result of the positional encodings are dropped and passed to the 1st, 2nd, & 3rd sub-layer through
residual connections. 

The masked multi-head attention layer (1st sub-layer) recieves the positional encodings and operates the same as the normal multi-head attention
layer (described above), but it masks positions to prevent positions from attenting to subsequent positions (i.e. it makes sure a position can 
only be predicted from the positions that preceed it). The result is normalized then passed to the unmasked multi-head attention
layer. 

The unmasked mutli-head attention (2nd sub-layer) layer recieves the result from the 1st sub-layer, but also the normalized result from the encoders
feed forward network. It then peforms the same computation as the multi-head attention layer described in the encoder. This result is 
normalized then fed into the feed forward network.

The feed forward network (3rd sub-layer) peforms the same operations as described in the encoder. The result is then normalzed and passed to 
a linear function so that softmax can be applied to compute the probabilities for the next predicted word.

Note: residual connections are used to prevent the vanishing gradient issue as gradients are back-propgated when learning

Defaults
d_model:
model defaults to 512 dimensions (less dimensions means less information passed between sub-layers with a gain of computation, 
while more means more information at a cost of compututation)

n_head:
defaults to 8 (scaled dot-product attention layers is proportional to number of heads. i.e. the amount of 
learnable linear projections is dependent on the number of heads)

n_encoder:
defaults to 6 and defines the amount of encoder stacks to be used in the transformer network

n_decoder:
defaults to 6 and defines the amount of decoder stacks to be used in the transformer network

feed_forward:
defaults to 2048 hidden dimensions (increasing dimensions may help improve fit of input to next multi-head attention layer)

dropout: 
defaults to 0.1 or 10%. Dropout helps prevent overfitting, but dropping too many inputs may prevent the network from learning
anything, while dropping not enough may cause the network to overfit
"""

class Transformer(nn.Module):

    # Constructor
    def __init__(self, n_tokens: int, d_model: int=512, n_head: int=8, n_encoder: int=6, n_decoder: int=6, feed_forward: int=2048, dropout: float=0.1):         
        super().__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        # transformer layers
        self.embedding = nn.Embedding(n_tokens, d_model)
        self.pos_encoder = PositionalEncoder(d_model=d_model, dropout=dropout, maxlen=5000)
        self.transformer = nn.Transformer(d_model=d_model, nhead=n_head, num_encoder_layers=n_encoder, 
                                        num_decoder_layers=n_decoder, dim_feedforward=feed_forward, dropout=dropout)
        self.out = nn.Linear(d_model, n_tokens)
    

    def forward(self, src: torch.LongTensor, tgt: torch.LongTensor, tgt_mask=None, src_pad_mask=None, tgt_pad_mask=None) -> torch.LongTensor:
        # src shape: (batch_size, src seq_length), tgt shape: (batch_size, tgt seq_length)

        # embedding + positional encoding shape: (batch_size, sequence length, d_model)
        src = self.embedding(src) * np.sqrt(self.d_model)
        tgt = self.embedding(tgt) * np.sqrt(self.d_model)
        src = self.pos_encoder(src)
        tgt = self.pos_encoder(tgt)
        
        # reshape: (seq_len, batch_size, d_model),
        src = src.permute(1,0,2)
        tgt = tgt.permute(1,0,2)

        # transformer inputs shape: (seq_len, batch_size, n_tokens)
        decoder_out = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)

        # compute linear function and apply softmax (Cross Entropy Loss already does this just return n_tokens as out)
        out = self.out(decoder_out)
        return out # shape: (n_tokens, batch_size, seq_len)
    
    # create mask for tgt
    def get_tgt_mask(self, seq_length: int) -> torch.LongTensor:
        # keep decoder from peeking ahead (i.e. show one word at a time in the sequence)
        mask = torch.tril(torch.ones(seq_length, seq_length) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        return mask
    
    # create binary encoded matrix ignore pad
    def create_pad_mask(self, matrix: torch.LongTensor, pad_val: int=0) -> torch.BoolTensor:
        # matrix = [3, 2, 1, 8, 0, 0, 0], pad_v = 0 -> [False, False, False, False, True, True, True]
        return (matrix == pad_val)

In [4]:
# generates dataloader of synthetic tokenized sequences
def generate_dataset(n_tokens, n_samples, minlen, maxlen, pad_val=0, batch_size=64):
    samples = []
    # create n samples
    for i in range(n_samples):
        seq_len = np.random.randint(minlen, maxlen + 1) # generate a sequence in a length range
        seq = np.random.randint(1, n_tokens + 1, seq_len) # create random tokens of that length

        # pad if too short
        if seq_len < maxlen:
            pad = np.zeros(maxlen - seq_len) + pad_val
            seq = np.append(seq, pad)
        samples.append(seq) # add to samples

    # creating dataloader
    samples = np.array(samples, dtype=np.int64)
    dataset = torch.utils.data.TensorDataset(torch.from_numpy(samples), torch.from_numpy(samples))
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
    return dataloader

In [5]:
# trains network 
def train(net, optimizer, loss_fn, dataloader, epochs=3, epoch_pct=0.25, device=None):
    # prepare network & metrics for training
    net.train()
    n = len(dataloader.dataset)
    m = len(dataloader)
    accum_loss = 0

    # train net over dataset for e epochs
    for epoch in range(epochs):
        samples_trained = 0
        # train over each batch
        for i, batch in enumerate(dataloader, 0):
            # get inputs and move to device
            inputs, labels = batch
            batch_size = inputs.size(0)
            src, tgt = inputs.to(device), labels.to(device)

            # create SOS and EOS tokens
            tgt_input, tgt_output = tgt[:, :-1], tgt[:, 1:]

            # mask out pad tokens and mask target input to keep decoder from cheating
            tgt_len = tgt_input.size(1)
            tgt_mask = net.get_tgt_mask(tgt_len).to(device)
            src_pad_mask = net.create_pad_mask(src, pad_val=0).to(device)
            tgt_pad_mask = net.create_pad_mask(tgt_input, pad_val=0).to(device)

            # compute prediction (softmax)
            pred = net(src, tgt_input, tgt_mask, src_pad_mask, tgt_pad_mask)
            pred = pred.permute(1, 2, 0) # reshape (batch_size, seq_len, n_tokens)

            # compute loss and backpropagate
            loss = loss_fn(pred, tgt_output)
            optimizer.zero_grad() # reset gradient
            loss.backward() # compute gradient
            optimizer.step() # update net parameters from gradient
            accum_loss += loss.item() # accumulate loss
            samples_trained += batch_size # keep track of sample trained
            
            # display info
            if (i + 1) % int(m * epoch_pct) == 0:
                print(f'{(i + 1) / m * 100:.0f}% of epoch completed | {samples_trained if samples_trained < n else n}/{n} samples trained | loss: {loss.item():.4f}')
        print(f'Epoch {epoch + 1} complete\n{n}/{n} samples trained | loss: {loss.item():.4f}')
    print(f'Training complete\navg loss: {accum_loss / (m * epochs):.4f}')

In [6]:
# tests transformer given testloader
def test(net, loss_fn, testloader, print_pct=0.25, device=None):
    # prepare network and metrics
    net.eval()
    accum_loss = 0
    correct = 0
    m = len(testloader)

    # don't compute new gradients
    with torch.no_grad():
        outputs_seen = 0

        # test each batch
        for i, batch in enumerate(testloader):

            # get the inputs and labels & move to device
            inputs, labels = batch
            src, tgt = inputs.to(device), labels.to(device)

            # shift to get SOS & EOS tokens 
            tgt_input, tgt_output = tgt[:, :-1], tgt[:, 1:]

            # mask padding for src & tgt, also mask tgt to prevent decoder from cheating
            tgt_len = tgt_input.size(1)
            tgt_mask = net.get_tgt_mask(tgt_len).to(device)
            src_pad_mask = net.create_pad_mask(src).to(device)
            tgt_pad_mask = net.create_pad_mask(tgt_input).to(device)
            
            # get prediction
            pred = net(src, tgt_input, tgt_mask, src_pad_mask, tgt_pad_mask) # tensor of likelihoods (softmax not applied)
            pred = pred.permute(1, 2, 0) # reshape: (batch_size, seq_len, n_tokens)
            loss = loss_fn(pred, tgt_output) # find loss (softmax applied)
            accum_loss += loss.item() # accumulate average batch loss

            # find accuracy
            batch_size = inputs.size(0)
            pred = torch.argmax(pred, dim=1) # get max val for each token vector
            correct += (pred == tgt_output).sum().item() # find where tokens match target output
            outputs_seen += batch_size * tgt_len # keep track of amount of tokens seen

            # display information
            if (i + 1) % (m * print_pct) == 0:
                print(f'{(i + 1) / m * 100:.0f}% of test completed | loss: {loss.item():.4f} | acc: {correct / outputs_seen:.4f}')
        print(f'Testing complete | avg loss: {accum_loss / m:.4f} | acc: {correct / outputs_seen:.4f}')

In [7]:
n_tokens = 5
n_samples = 10000
minlen = 15
maxlen = 30
batch_size = 64

trainloader = generate_dataset(n_tokens, n_samples, minlen, maxlen, batch_size=batch_size)
dataiter = iter(trainloader)
batch = next(dataiter)

inputs, labels = batch
print(f'Batch shape: {inputs.size()}\nExample source: {inputs[0]}\nShifted target input: {labels[0][:-1]}\nShifted target output: {labels[0][1:]}')

Batch shape: torch.Size([64, 30])
Example source: tensor([3, 5, 3, 1, 3, 4, 1, 2, 4, 2, 4, 3, 2, 4, 3, 2, 3, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])
Shifted target input: tensor([3, 5, 3, 1, 3, 4, 1, 2, 4, 2, 4, 3, 2, 4, 3, 2, 3, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0])
Shifted target output: tensor([5, 3, 1, 3, 4, 1, 2, 4, 2, 4, 3, 2, 4, 3, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0])


In [8]:
net = Transformer(n_tokens=n_tokens + 1, d_model=512, n_head=2, n_encoder=3, n_decoder=3, feed_forward=2048, dropout=0.1)
optimizer = optim.SGD(net.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Training on {"gpu" if device == torch.device("cuda") else "cpu"}')
net.to(device);

Training on gpu


In [9]:
train(net, optimizer, loss_fn, trainloader, epochs=1, device=device)

25% of epoch completed | 2496/10000 samples trained | loss: 1.3325
50% of epoch completed | 4992/10000 samples trained | loss: 1.3114
75% of epoch completed | 7488/10000 samples trained | loss: 1.2768
99% of epoch completed | 9984/10000 samples trained | loss: 1.2123
Epoch 1 complete
10000/10000 samples trained | loss: 1.2537
Training complete
avg loss: 1.3321


In [10]:
# making fake testset
n_samples = 2000
testloader = generate_dataset(n_tokens, n_samples, minlen, maxlen, batch_size=batch_size)

In [11]:
test(net, loss_fn, testloader, device=device)

25% of test completed | loss: 1.2605 | acc: 0.4611
50% of test completed | loss: 1.2100 | acc: 0.4620
75% of test completed | loss: 1.2307 | acc: 0.4658
100% of test completed | loss: 1.1932 | acc: 0.4654
Testing complete | avg loss: 1.2512 | acc: 0.4654


In [12]:
def predict(net: Transformer, inputs: np.array, pred_to: int, device=None):    
    predictions = []

    # iterate each tokenized sequence
    for src in inputs:
        src = np.array([src]) # reshape: (batch_size, seq_len)
        seq_len = len(src[0]) # seq_len
        tgt_input = [[np.squeeze(src)[0]]] if seq_len > 1 else src.copy() # get SOS (start of sentence)
        src = torch.tensor(src, dtype=torch.long, device=device) # convert to tensor
        tgt_input = torch.tensor(tgt_input, dtype=torch.long, device=device) # convert to tensor

        # predict next 'pred_to' tokens
        for i in range(pred_to):
            # get the mask to keep decoder from cheating
            tgt_mask = net.get_tgt_mask(tgt_input.size(1)).to(device)

            pred = net(src, tgt_input, tgt_mask) # compute prediction
            token = pred.topk(1)[1].view(-1)[-1].item() # get next token of highest probability
            
            token = torch.tensor([[token]], dtype=torch.long, device=device) # reshape (batch_size, seq_len)
            tgt_input = torch.cat((tgt_input, token), dim=1) # combine with previous tgt_input

        # add tokenized predicted sequence to predictions 
        predictions.append(tgt_input[:, 1:].to('cpu').numpy().squeeze()) # reshape: (seq_len)
    return predictions

In [13]:
# fake inputs generated
inputs = [np.random.randint(1, n_tokens + 1, size=np.random.randint(1, 6)) for i in range(10)]
inputs = np.array(inputs) # convert to np array
predictions = predict(net, inputs, pred_to=5, device=device) # get prediction
# show what transformer thinks next 5 tokens will be based on input sequence
for og, pred in zip(inputs, predictions):
    print(f'{og} -> {pred}')

/tmp/ipykernel_281961/2603079193.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  inputs = np.array(inputs) # convert to np array


[3 3 2] -> [3 3 3 3 3]
[1] -> [1 1 1 1 1]
[4 3 3 2 3] -> [3 3 3 3 3]
[3 3 2 5 3] -> [3 3 3 3 3]
[2 1 3 4 3] -> [3 4 3 3 3]
[5 3 3 1] -> [3 3 3 3 3]
[2 4 5 3 2] -> [2 2 2 2 2]
[1 2 3] -> [2 2 3 2 3]
[5 3 2] -> [2 2 5 2 2]
[1 4 3 3] -> [3 3 3 3 3]
